In [ ]:
import pandas as pd
import torch
import torch.nn as nn
from transformers import GPT2Tokenizer, GPT2Model, GPT2LMHeadModel
import torch
from transformers import RobertaConfig, RobertaModel, RobertaTokenizer, RobertaModel
import math

In [ ]:
go_emotions_train = pd.read_csv('train.tsv.txt', sep='\t', header = None)

In [ ]:
go_emotions_train

,0,1,2
0,My favourite food is anything I didn't have to...,27,eebbqej
1,"Now if he does off himself, everyone will thin...",27,ed00q6i
2,WHY THE FUCK IS BAYLESS ISOING,2,eezlygj
3,To make her feel threatened,14,ed7ypvh
4,Dirty Southern Wankers,3,ed0bdzj
...,...,...,...
43405,Added you mate well I’ve just got the bow and ...,18,edsb738
43406,Always thought that was funny but is it a refe...,6,ee7fdou
43407,What are you talking about? Anything bad that ...,3,efgbhks
43408,"More like a baptism, with sexy results!",13,ed1naf8


In [ ]:
go_emotions_test = pd.read_csv('test.tsv.txt', sep='\t', header = None)

In [ ]:
go_emotions_dev = pd.read_csv('dev.tsv.txt', sep='\t', header = None)

In [ ]:
from transformers import RobertaConfig, RobertaModel, RobertaTokenizer, RobertaModel

In [ ]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"

In [ ]:
t = torch.cuda.get_device_properties(0).total_memory
r = torch.cuda.memory_reserved(0)
a = torch.cuda.memory_allocated(0)
f = r-a  # free inside reserved
print(t/1000000000)
print(r)
print(a)
print(f)

42.314694656
0
0
0


In [ ]:
R_tokenizer = RobertaTokenizer.from_pretrained("roberta-large")
Roberta_model = RobertaModel.from_pretrained("roberta-large").to(device)

Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
# t = torch.cuda.get_device_properties(0).total_memory
# r = torch.cuda.memory_reserved(0)
# a = torch.cuda.memory_allocated(0)
# f = r-a  # free inside reserved
# print(t/1000000000)
# print(r/1000000000)
# print(a/1000000000)
# print(f/1000000000)

In [ ]:
# emotion_train_roberta_embeddings = []
# with torch.no_grad():
#     for idx, val in enumerate(go_emotions_train.values):
#         R_tokenized = R_tokenizer(val[0], return_tensors = 'pt', truncation=True).to(device)
#         #print(R_tokenized.shape)
#         R_embed = Roberta_model(**R_tokenized).last_hidden_state.squeeze()
#         print(R_embed.shape)
#         del R_tokenized
#         R_embed_cpu = R_embed.to('cpu')
#         del R_embed
#         emotion_train_roberta_embeddings.append(R_embed_cpu)
#         if idx%1000 == 0:
#             t = torch.cuda.get_device_properties(0).total_memory
#             r = torch.cuda.memory_reserved(0)
#             a = torch.cuda.memory_allocated(0)
#             f = r-a  # free inside reserved
#             print(t/1000000000)
#             print(r/1000000000)
#             print(a/1000000000)
#             print(f/1000000000)
#             print(idx)
#         break
#         #print(val[0])
#         #print(type(R_tokenized))
        
    

In [ ]:
emotion_dev_roberta_embeddings = []
with torch.no_grad():
    for idx, val in enumerate(go_emotions_dev.values):
        R_tokenized = R_tokenizer(val[0], return_tensors = 'pt', truncation=True).to(device)
        #print(R_tokenized.shape)
        R_embed = Roberta_model(**R_tokenized).last_hidden_state.squeeze()
        #print(R_embed.shape)
        del R_tokenized
        R_embed_cpu = R_embed.to('cpu')
        del R_embed
        emotion_dev_roberta_embeddings.append(R_embed_cpu)
        if idx%1000 == 0:
            t = torch.cuda.get_device_properties(0).total_memory
            r = torch.cuda.memory_reserved(0)
            a = torch.cuda.memory_allocated(0)
            f = r-a  # free inside reserved
            print(t/1000000000)
            print(r/1000000000)
            print(a/1000000000)
            print(f/1000000000)
            print(idx)
        #print(val[0])
        #print(type(R_tokenized))
        

42.314694656
1.43654912
1.421448192
0.015100928
0
42.314694656
1.43654912
1.421448192
0.015100928
1000
42.314694656
1.43654912
1.421448192
0.015100928
2000
42.314694656
1.43654912
1.421448192
0.015100928
3000
42.314694656
1.438646272
1.421448192
0.01719808
4000
42.314694656
1.438646272
1.421448192
0.01719808
5000


In [ ]:
# print(emotion_train_roberta_embeddings[0].shape)

In [ ]:
gpt_tokenizer = GPT2Tokenizer.from_pretrained("gpt2-xl")
gpt_head_model = GPT2LMHeadModel.from_pretrained('gpt2-xl').to(device)
gpt_head_transformer = gpt_head_model.transformer

In [ ]:
emotion_dev_gpt_logits = []
with torch.no_grad():
    for idx, val in enumerate(go_emotions_dev.values):
        gpt_tokenized = gpt_tokenizer(val[0], return_tensors = 'pt', truncation=True).to(device)
        gpt_embed = gpt_head_transformer(**gpt_tokenized).last_hidden_state.squeeze()
        del gpt_tokenized
        gpt_embed_cpu = gpt_embed.to('cpu')
        del gpt_embed
        emotion_dev_gpt_logits.append(gpt_embed_cpu)
        if idx%1000 == 0:
            t = torch.cuda.get_device_properties(0).total_memory
            r = torch.cuda.memory_reserved(0)
            a = torch.cuda.memory_allocated(0)
            f = r-a  # free inside reserved
            print(t/1000000000)
            print(r/1000000000)
            print(a/1000000000)
            print(f/1000000000)
            print(idx)

42.314694656
7.868514304
7.84389376
0.024620544
0
42.314694656
7.90626304
7.84389376
0.06236928
1000
42.314694656
7.90626304
7.84389376
0.06236928
2000
42.314694656
7.90626304
7.84389376
0.06236928
3000
42.314694656
7.9691776
7.84389376
0.12528384
4000
42.314694656
7.99014912
7.84389376
0.14625536
5000


In [ ]:
torch.save(emotion_dev_roberta_embeddings, 'emo_roberta_dev.pt')
torch.save(emotion_dev_gpt_logits, 'emo_gpt2_dev.pt')

In [ ]:
# print(len(emotion_train_gpt_logits))
# print(len(emotion_train_roberta_embeddings))
# print(emotion_train_gpt_logits[0].shape)
# print(emotion_train_roberta_embeddings[0].shape)

In [ ]:
# torch.save(emotion_train_gpt_logits, 'emo_gpt2-xl.pt')

In [ ]:
# torch.save(emotion_train_roberta_embeddings, 'emo_Roberta.pt')

In [ ]:
# t = torch.cuda.get_device_properties(0).total_memory
# r = torch.cuda.memory_reserved(0)
# a = torch.cuda.memory_allocated(0)
# f = r-a  # free inside reserved
# print(t/1000000000)
# print(r/1000000000)
# print(a/1000000000)
# print(f/1000000000)


In [ ]:
# del gpt_head_model
# del Roberta_model
# del gpt_head_transformer

In [ ]:
# import gc

# gc.collect()
# torch.cuda.empty_cache()

In [ ]:
# t = torch.cuda.get_device_properties(0).total_memory
# r = torch.cuda.memory_reserved(0)
# a = torch.cuda.memory_allocated(0)
# f = r-a  # free inside reserved
# print(t/1000000000)
# print(r/1000000000)
# print(a/1000000000)
# print(f/1000000000)


In [ ]:
# from transformers import GPT2Tokenizer, GPTNeoModel, GPTNeoForCausalLM
# gpt_neo_tokenizer =GPT2Tokenizer.from_pretrained("EleutherAI/gpt-neo-2.7B")
# gpt_neo_head_transformer = GPTNeoForCausalLM.from_pretrained("EleutherAI/gpt-neo-2.7B").transformer.to(device)

In [ ]:
# from transformers import GPT2Tokenizer, GPTNeoModel, GPTNeoForCausalLM
# gpt_neo_tokenizer =GPT2Tokenizer.from_pretrained("EleutherAI/gpt-neo-1.3B")
# gpt_neo_head_transformer = GPTNeoForCausalLM.from_pretrained("EleutherAI/gpt-neo-1.3B").transformer.to(device)

In [ ]:
# del gpt_neo_head_model
# del gpt_neo_head_transformer

In [ ]:
# print(len(emotion_train_neo_logits))
# print(emotion_train_neo_logits[0].shape)

In [ ]:
# t = torch.cuda.get_device_properties(0).total_memory
# r = torch.cuda.memory_reserved(0)
# a = torch.cuda.memory_allocated(0)
# f = r-a  # free inside reserved
# print(t/1000000000)
# print(r/1000000000)
# print(a/1000000000)
# print(f/1000000000)


In [ ]:
# emotion_train_neo_logits = []
# with torch.no_grad():
#     for idx, val in enumerate(go_emotions_train.values):
#         neo_tokenized = gpt_neo_tokenizer(val[0], return_tensors = 'pt', truncation=True).to(device)
#         neo_embed = gpt_neo_head_transformer(**neo_tokenized).last_hidden_state.squeeze()
#         del neo_tokenized
#         neo_embed_cpu = neo_embed.to('cpu')
#         del neo_embed
#         emotion_train_neo_logits.append(neo_embed_cpu)
#         if idx%1000 == 0:
#             t = torch.cuda.get_device_properties(0).total_memory
#             r = torch.cuda.memory_reserved(0)
#             a = torch.cuda.memory_allocated(0)
#             f = r-a  # free inside reserved
#             print(t/1000000000)
#             print(r/1000000000)
#             print(a/1000000000)
#             print(f/1000000000)
#             print(idx)

In [ ]:
# torch.save(emotion_train_neo_logits, 'emo_neo1_3.pt')